In [1]:
import webbrowser
import requests
import bs4
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import numpy as np
import time
import os, sys
from stat import S_ISREG, ST_CTIME, ST_MODE
import shutil
from zipfile import ZipFile

# Set up browser
browser = webdriver.Chrome(executable_path=r"/Users/DanielVassallo/chromedriver.exe")
browser.get('https://www.oddsshark.com/mlb/odds')


In [2]:
leftCol = browser.find_element_by_class_name('op-left-column-wrapper')
topTeams = leftCol.find_elements_by_xpath('.//div[@class="op-team-data-wrapper not-futures"]/div[@class="op-matchup-wrapper baseball"]/div[@class="op-matchup-right"]/div[@class="op-matchup-team op-matchup-text op-team-top"]')
bottomTeams = leftCol.find_elements_by_xpath('.//div[@class="op-team-data-wrapper not-futures"]/div[@class="op-matchup-wrapper baseball"]/div[@class="op-matchup-right"]/div[@class="op-matchup-team op-matchup-text op-team-bottom"]')

oddsList = browser.find_element_by_id('op-results')


In [3]:
for num, team in enumerate(topTeams):
    teamName = eval(team.get_attribute('data-op-name'))
    if teamName['short_name']=='NYY':
        gameNum = num+1 # row of game
        teamNum=1 # away team
        homeShort = teamName['short_name']
        homeLong = teamName['full_name']
        Text = 'Away Team: '+homeLong+' ('+homeShort+')'
        
        # Get Away Team
        awayShort = eval(bottomTeams[num].get_attribute('data-op-name'))['short_name']
        awayLong = eval(bottomTeams[num].get_attribute('data-op-name'))['full_name']
        Text2 = 'Home Team: '+awayLong+' ('+awayShort+')'
        print('Game '+str(gameNum))
        print(Text2)
        print(Text)
        break
        
for num, team in enumerate(bottomTeams):
    teamName = eval(team.get_attribute('data-op-name'))
    if teamName['short_name']=='NYY':
        gameNum = num+1 # row of game
        teamNum=2 # home team
        awayShort = teamName['short_name']
        awayLong = teamName['full_name']
        Text = 'Home Team: '+awayLong+' ('+awayShort+')'
        
        # Get Home Team
        homeShort = eval(topTeams[num].get_attribute('data-op-name'))['short_name']
        homeLong = eval(topTeams[num].get_attribute('data-op-name'))['full_name']
        Text2 = 'Away Team: '+homeLong+' ('+homeShort+')'
        print('Game '+str(gameNum))
        print(Text)
        print(Text2)
        break
        

Game 2
Home Team: NY Yankees (NYY)
Away Team: Tampa Bay (TB)


In [4]:
t1Odds = '//*[@id="op-results"]/div['+str(gameNum*2)+']/div/div[1]'
t2Odds = '//*[@id="op-results"]/div['+str(gameNum*2)+']/div/div[3]'
topOdds = oddsList.find_elements_by_xpath(t1Odds)
bottomOdds = oddsList.find_elements_by_xpath(t2Odds)


In [5]:
awayOdds = []
homeOdds = []
OverUnder = []
for odds in topOdds:
    if eval(odds.get_attribute('data-op-moneyline'))['fullgame']:
        awayOdds.append(eval(odds.get_attribute('data-op-moneyline'))['fullgame'])
        OverUnder.append(eval(odds.get_attribute('data-op-total'))['fullgame'])
awayOdds = np.array(awayOdds).astype(int)
OverUnder = np.array(OverUnder).astype(float)
    
for odds in bottomOdds:
    if eval(odds.get_attribute('data-op-moneyline'))['fullgame']:
        homeOdds.append(eval(odds.get_attribute('data-op-moneyline'))['fullgame'])
homeOdds = np.array(homeOdds).astype(int)


In [6]:
FinalAway = int(np.round(np.mean(awayOdds/5))*5)
if FinalAway>0:
    FinalAway = '+'+str(FinalAway)
else:
    FinalAway = str(FinalAway)
FinalHome = int(np.round(np.mean(homeOdds/5))*5)
if FinalHome>0:
    FinalHome = '+'+str(FinalHome)
else:
    FinalHome = str(FinalHome)
FinalOU = str(np.median(OverUnder))


In [7]:
T1 = 'Odds: \n'+awayLong+': '+FinalAway+'\n'+homeLong+': '+FinalHome+'\n'+'O/U: '+FinalOU
print(T1)


Odds: 
NY Yankees: +145
Tampa Bay: -155
O/U: 10.0
